#### Power iterations for symmetric matrices

So far we know that for `diagonalizable` $A\in \mathbf{R}^{n \times n}$, the method of power iterations finds the dominant eigenvalue and corresponding eigenvector through the following

* starting from $x^0$
* compute $y^k=Ax^{k-1}$
* get new $x^{k}$ by normalizing $y^k$ ($l_2$ norm, infinity norm, etc)
$$x^{k}=\frac{y^k}{\|y^k\|}$$

For which $x^k\rightarrow v_1$, and Rayleigh quotient $\frac{(x^k)^TAx^k}{(x^k)^Tx^k}\rightarrow \lambda_1$

For `symmetric` matrices, we can also update using

$$A\leftarrow A - \lambda_1 v_1 v_1^T$$

and progressively find remaining eigenvalues and eigenvectors, due to the fact that symmetric matrices have orthonormal eigenvectors

#### General eigenvalues problems for nonsymmetric matrices

If matrix $A$ is `nonsymmetric`, we know that we can choose right eigenvector $v$ and left eigenvector $u$ such that

$$u_i^Tv_j = \left\{\begin{array}{rcl}0 &i\neq j \\1 &i=j \end{array}\right.$$

Update to matrix $A$ after getting $\lambda_1, v_1$ can be expressed as

$$A\leftarrow A - \lambda_1 v_1 u_1^T$$

Similarly, what we want is that the updated $A$ still has all remaining $\lambda_i, v_i, i>1$ while making $v_1$ corresponding to a zero eigenvalue, and we can again verify both

$$\begin{align*}
&(A - \lambda_1 v_1 u_1^T)v_1 \\
&=Av_1-\lambda_1v_1(u_1^Tv_1)\\
& u_1^Tv_1=1 \\
&=Av_1-\lambda_1v_1\\
&=0
\end{align*}$$

and

$$\begin{align*}
&(A - \lambda_1 v_1 u_1^T)v_i \\
&=Av_i-\lambda_1v_1(u_1^Tv_i)\\
& u_1^Tv_i=0,\forall i\neq 1 \\
&=Av_i\\
&=\lambda_iv_i
\end{align*}$$

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(formatter={'float': '{: 0.4f}'.format})

plt.style.use('dark_background')
# color: https://matplotlib.org/stable/gallery/color/named_colors.htm

In [ ]:
def power_iteration_nonsymmetric(A, num_eigen, num_iter=10000, converge_tol=1e-7, eigen_tol=1e-7):
    n = A.shape[0]
    eigenvalues = []
    right_eigenvectors = []
    left_eigenvectors = []
    A_current = A.copy()

    for k in range(num_eigen):
        v = np.random.rand(n)
        v /= np.linalg.norm(v)
        u = np.random.rand(n)
        u /= np.linalg.norm(u)

        for j in range(num_iter):
            # Right eigenvector
            y = A_current @ v
            norm_y = np.linalg.norm(y)
            if norm_y < eigen_tol:
                print(f"Norm of (#{k+1}) right eigenvector is too small, stopping iteration")
                break
            v_next = y / norm_y

            # Left eigenvector
            z = A_current.T @ u
            norm_z = np.linalg.norm(z)
            if norm_z < eigen_tol:
                print(f"Norm of (#{k+1}) left eigenvector is too small, stopping iteration")
                break
            u_next = z / norm_z

            # Check convergence
            if np.linalg.norm(v_next - v) < converge_tol and np.linalg.norm(u_next - u) < converge_tol:
                break

            v = v_next
            u = u_next

        if norm_y < eigen_tol or norm_z < eigen_tol:
            continue

        print(f'{j+1} iterations for eigenvalue #{k+1}')

        # Normalize u and v such that u^Tv = 1
        scaling = u @ v
        if abs(scaling) < eigen_tol:
            print(f"Scaling factor u^Tv is too small, cannot normalize")
            continue
        v = v / scaling

        # Rayleigh quotient using right eigenvectors
        eigenvalue = v @ (A_current @ v) / (v @ v)

        if abs(eigenvalue) < eigen_tol:
            print(f"Eigenvalue {eigenvalue} is too small and is ignored")
            continue

        eigenvalues.append(eigenvalue)
        right_eigenvectors.append(v)
        left_eigenvectors.append(u)

        A_current = A_current - eigenvalue * np.outer(v, u)

    if not eigenvalues:
        print("No valid eigenvalues found")
        return np.array([]), np.array([]), np.array([])

    return np.array(eigenvalues), np.column_stack(right_eigenvectors), np.column_stack(left_eigenvectors)

In [ ]:
def diagonalizable_mat(n):
    # Create diagonal matrix D with eigenvalues
    D = np.diag(np.concatenate((200*np.random.rand(n//2)-100, 0.1*np.random.rand(n-n//2))))

    # Generate a random invertible matrix
    P = np.random.rand(n, n)
    while np.linalg.cond(P) > 1e8:  # Check conditioning
        P = np.random.rand(n, n)

    # Use similarity transformation to create diagonalizable, but nonsymmetric matrix
    return P @ D @ np.linalg.inv(P)

In [ ]:
np.random.seed(50)

A_size = 8
A = diagonalizable_mat(A_size)

eigenvalues, right_eigenvectors, left_eigenvectors = power_iteration_nonsymmetric(A, A_size)

# Normalize right eigenvectors and adjust left eigenvectors
for i in range(right_eigenvectors.shape[1]):
    v = right_eigenvectors[:, i]
    u = left_eigenvectors[:, i]
    norm_v = np.linalg.norm(v)
    if norm_v < 1e-6:
        print(f"Norm of right eigenvector #{i+1} is too small, cannot normalize")
        continue
    # To be consistent with NumPy
    v_normalized = v / norm_v
    # To ensure biorthogonality
    u_adjusted = u * norm_v

    right_eigenvectors[:, i] = v_normalized
    left_eigenvectors[:, i] = u_adjusted

    print(f"u_{i+1}^T v_{i+1} = {v_normalized @ u_adjusted:.4f}")

print("\nComputed eigenvalues")
for idx, eigenvalue in enumerate(eigenvalues, 1):
    print(f"# {idx}: {eigenvalue:.4f}")

print("\nComputed right eigenvectors (columns)")
print(right_eigenvectors)

print("\nComputed left eigenvectors (columns)")
print(left_eigenvectors)

# Compare with NumPy's outcomes
eigenvalues_np, right_eigenvectors_np = np.linalg.eig(A)
print("\nEigenvalues from NumPy")
print(eigenvalues_np)

print("\nRight eigenvectors from NumPy")
print(right_eigenvectors_np)

# Reconstruct A using left/right eigenvectors
A_reconstructed = right_eigenvectors @ np.diag(eigenvalues) @ left_eigenvectors.T

print("\nA reconstructed")
print(A_reconstructed)

print("\nOriginal A")
print(A)

# Check error
reconstruction_error = np.linalg.norm(A - A_reconstructed)
print(f"\nReconstruction error: {reconstruction_error}")

10000 iterations for eigenvalue #1
10000 iterations for eigenvalue #2
10000 iterations for eigenvalue #3
10000 iterations for eigenvalue #4
57 iterations for eigenvalue #5
28 iterations for eigenvalue #6
168 iterations for eigenvalue #7
3 iterations for eigenvalue #8
u_1^T v_1 = 1.0000
u_2^T v_2 = 1.0000
u_3^T v_3 = 1.0000
u_4^T v_4 = 1.0000
u_5^T v_5 = 1.0000
u_6^T v_6 = 1.0000
u_7^T v_7 = 1.0000
u_8^T v_8 = 1.0000

Computed eigenvalues
# 1: -54.3834
# 2: -48.9052
# 3: -20.7340
# 4: -1.0797
# 5: 0.0997
# 6: 0.0772
# 7: 0.0408
# 8: 0.0377

Computed right eigenvectors (columns)
[[ 0.1853  0.2079  0.2574  0.3834  0.5393  0.3564  0.4128  0.0885]
 [ 0.5309  0.4047  0.2861  0.1581  0.2906  0.3637  0.4213  0.3085]
 [ 0.0312  0.1962  0.0413  0.3369  0.5134  0.4545  0.4259  0.1094]
 [ 0.2760  0.5588  0.2974  0.3694  0.3603  0.1413  0.2719  0.4155]
 [ 0.5249  0.4780  0.3161  0.3440  0.4335  0.0795  0.3295  0.5583]
 [ 0.2318  0.1769  0.4512  0.4619  0.1416  0.1137  0.3809  0.2846]
 [ 0.3945  0.2